# Aplicación para hacer Web Scrapping de un sitio web y utilización de LLM para resumen y puntos clave

The Scenario
Our company's leadership team, working in the agri-business sector, wants to understand the key dynamics of the global produce trade to identify potential market opportunities. They want a data-driven snapshot of the topics and trends being discussed across several key business areas.
Your mission is to analyze articles from the fresh produce industry's main resources page, extract key information, use modern AI tools to enrich the data, and present your findings in a clear and concise manner.

Español:
El escenario
El equipo directivo de nuestra empresa, que trabaja en el sector agroalimentario, quiere comprender la dinámica clave del comercio mundial de productos agrícolas para identificar posibles oportunidades de mercado. Quieren una instantánea basada en datos de los temas y tendencias que se debaten en varias áreas de negocio clave.
Tu misión consiste en analizar artículos de la página principal de recursos del sector de los productos frescos, extraer información clave, utilizar modernas herramientas de IA para enriquecer los datos y presentar tus conclusiones de forma clara y concisa.

## Web Scrapping

### Extracción de subdominios o links de artículos de las páginas principales.

Primero se va a extraer el contenido de una página web para después generar el algoritmo para extraer los subdominios para en un posterior paso extraer su contenido

URLS a extraer:
- https://www.freshproduce.com/resources/global-trade/
- https://www.freshproduce.com/resources/technology/
- https://www.freshproduce.com/resources/food-safety/

#### Contenido estático

In [1]:
primera_url_a_extraer_subdominio_ejemplo = "https://www.freshproduce.com/resources/global-trade/"

In [2]:
import requests
from bs4 import BeautifulSoup

def scrape_website(url):
    # Realiza web scraping en la URL proporcionada. Extrae todo el HTML.

    try:
        print(f"Extrayendo HTML de: {url}")
        response = requests.get(url) # Realizar una solicitud HTTP GET a la URL
        response.raise_for_status()  # Lanza un error para códigos de estado HTTP incorrectos (4xx o 5xx)

        # 2. Convierte el contenido obtenido a una estructura HTML con BeautifulSoup
        soup_html = BeautifulSoup(response.text, 'html.parser')
        print("Contenido HTML parseado exitosamente.")
        return soup_html

    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con la URL: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")



In [3]:
Extraccion_HTML = scrape_website(primera_url_a_extraer_subdominio_ejemplo)

Extrayendo HTML de: https://www.freshproduce.com/resources/global-trade/
Contenido HTML parseado exitosamente.


Haciendo inspección del sitio web, hay dos clases de etiqueta tipo div necesarias para una correcta inspección:
- search-stats: permite saber el número total de elementos que contiene la página y el número de elementos recorridos
- result-panel: contiene todos los resultados de los artículos, con su respectivo subdominio.

In [4]:
etiquetas_a_extraer = ['search-stats', 'result-panel']
for etiqueta in etiquetas_a_extraer:
    elementos = Extraccion_HTML.find_all(class_=etiqueta)
    print(elementos)

[]
[]


En el paso anterior se puede observar que no encontró ninguna de las etiquetas a buscar. Muy seguramente porque sea contenido dinámico.

#### Contenido dinámico

Se utilizará Selenium apoyado con Beautiful Soup para casos de contenido dinámico.

In [5]:
from selenium import webdriver #Controlar google chrome para obtener HTML con contenido dinámico
from selenium.webdriver.chrome.options import Options #Controlar google chrome sin interfaz gráfica
from selenium.webdriver.support.ui import WebDriverWait  # Para esperar elementos dinámicos
from selenium.webdriver.support import expected_conditions as EC  # Condiciones para la espera
from selenium.webdriver.common.by import By  # Para localizar elementos (por clase, ID, etc.)
from bs4 import BeautifulSoup # #Para parsear el HTML obtenido con selenium
import chromedriver_autoinstaller #Instalación automática del driver de Chrome

def scrape_website_with_dynamic_content(page, classes_to_wait = None, ID_OR_CLASS='CLASS'):
    # Instala automáticamente el chromedriver compatible con la versión de Chrome instalada
    chromedriver_autoinstaller.install() 

    options = Options() # Configuraciones de opciones para el navegador
    options.add_argument("--headless") # Ejecuta chrome sin interfaz gráfica

    #Necesario por falta de permisos en el entorno aislado del contenedor
    options.add_argument("--no-sandbox") #Desactiva seguridad de Chrome por problemas de permisos en algunos sistemas
    
    options.add_argument("--disable-dev-shm-usage") #Prevenir fallos por falta de memoria (RAM) compartida en algunos sistemas. Usa memoria de disco duro.

    driver = webdriver.Chrome(options=options) # Inicia navegador chrome con las opciones configuradas

    driver.get(page) # Carga la página web en el navegador

    
    if classes_to_wait:
        tiempo_espera = 30 # Tiempo máximo de espera en segundos para que se cargue el contenido dinámico
        wait = WebDriverWait(driver, tiempo_espera)
        if ID_OR_CLASS.upper() == 'CLASS':
            for class_to_wait in classes_to_wait: #Recorre cada clase a esperar
                wait.until(
                    EC.presence_of_element_located((By.CLASS_NAME, class_to_wait)) # Cambio a busqueda por clase
                )# Espera cada una de las clases
        elif ID_OR_CLASS.upper() == 'ID':
            for id_to_wait in classes_to_wait: #Recorre cada ID a esperar
                wait.until(
                    EC.presence_of_element_located((By.ID, id_to_wait)) #Cambio a busqueda por ID
                )# Espera cada uno de los IDs
        else:
            raise ValueError("ID_OR_CLASS debe ser 'CLASS' o 'ID'.")  # Manejo de error si el parámetro no es válido



    html_completo = driver.page_source  # Obtención del HTML renderizado completo

    # Usar BeautifulSoup para parsear
    soup = BeautifulSoup(html_completo, "html.parser")
    # Por ejemplo, buscar la tabla con clase 'search-results'
    driver.quit() #Cerrar navegador de Chrome
    return soup

In [6]:
Extraccion_HTML_dinamico = scrape_website_with_dynamic_content(primera_url_a_extraer_subdominio_ejemplo, classes_to_wait=etiquetas_a_extraer, ID_OR_CLASS='CLASS')

Verificamos que las etiquetas ahora se encuentren en el contenido.

In [7]:
for etiqueta in etiquetas_a_extraer:
    elementos = Extraccion_HTML_dinamico.find_all(class_=etiqueta)
    print(elementos)
    print(f"Cantidad de elementos encontrados con la clase '{etiqueta}': {len(elementos)}")

[<div class="search-stats"><p>Showing 1 - 9 of 23 results </p></div>]
Cantidad de elementos encontrados con la clase 'search-stats': 1
[<div class="result-panel"><div class="tile eventlandingpage theme-agave tile-grid"><div class="image-wrapper bg-agave"><img alt="Nighttime skyline in Sao Paulo Brazil " loading="eager" src="https://www.freshproduce.com/siteassets/images/events/brazil-conference/sao-paulo-skyline-at-night.jpg/_croppings/event-landing-page-grid.jpg"/></div><div class="caption"><p class="eyebrow">August 6-7, 2025 | São Paulo, Brazil</p><p class="title">The Brazil Conference &amp; Expo</p><p class="description">The Brazil Conference &amp; Expo the meeting point for retail buyers, executives &amp; industry leaders dedicated to the fresh produce supply chain in Brazil.</p><div class="cta-area"><a class="score-button seed-link" href="/events/the-brazil-conference/" target="_self">learn more</a></div></div></div><div class="tile resourcedetailpage theme-mint tile-grid"><div cl

Se encontraron los datos a buscar. Ahora se manipularán los datos de las etiquetas a necesitar para sus debidos datos: 'search-stats' y 'result-panel'

#### Manipulación de etiquetas para obtención de datos concretos
En la página web se requieren dos datos:
- Search-stats: Para hacer scrap de **todos** los artículos que contenga el dominio o subdominio y obtener de cada uno la etiqueta result-panel
- result-panel: Los subdominios que, juntándolo con el dominio principal, se logra recopilar el artículo original

##### **search-stats** 
Se extraen los elementos de búsqueda como el número de artículo que contiene actualmente la página, y el número de artículos disponible

In [8]:
etiqueta_search_stats = 'search-stats' # Definida en este apartado para evitar buscarla arriba en el código
search_stats = Extraccion_HTML_dinamico.find(class_=etiqueta_search_stats)  # Buscar el primer elemento con la clase 'search-stats'
if search_stats:
    print("Contenido de 'search-stats':")
    print(search_stats.text)
    print(search_stats.text.split())

Contenido de 'search-stats':
Showing 1 - 9 of 23 results 
['Showing', '1', '-', '9', 'of', '23', 'results']


Al separar todas las palabras hay 3 palabras que nos interesan:
- El último elemento de la página web ('9')
- El número de elementos de la página web ('23')
- Un punto de referencia entre ambos ('of')

Extraeremos cada uno de ellos

In [9]:
search_stats_arreglo = search_stats.text.split() 
print(search_stats_arreglo)
indice_palabra_of = search_stats_arreglo.index("of") #Desde qué elemento está el punto de referencia
print(f"Índice de la palabra 'of': {indice_palabra_of}")
ultimo_elemento_de_pagina = int(search_stats_arreglo[indice_palabra_of -1]) # La palabra anterior a "of" es el último elemento de la página
print(f"Último elemento de la página: {ultimo_elemento_de_pagina}")
total_elementos_de_pagina = int(search_stats_arreglo[indice_palabra_of + 1]) # La palabra siguiente a "of" es el total de elementos de la página
print(f"Total de elementos de la página: {total_elementos_de_pagina}")

['Showing', '1', '-', '9', 'of', '23', 'results']
Índice de la palabra 'of': 4
Último elemento de la página: 9
Total de elementos de la página: 23


Con esto se puede hacer una función que verifique si se ha recorrido todo el contenido.

In [10]:
# Verifica si la página HTML indica que es la última página de resultados.
def bool_is_final_page(HTML, etiqueta='search-stats'):
    search_stats = HTML.find(class_=etiqueta)
    if search_stats:
        search_stats_arreglo = search_stats.text.split()
        indice_palabra_of = search_stats_arreglo.index("of")
        ultimo_elemento_de_pagina = int(search_stats_arreglo[indice_palabra_of - 1])
        total_elementos_de_pagina = int(search_stats_arreglo[indice_palabra_of + 1])
        return (ultimo_elemento_de_pagina >= total_elementos_de_pagina) #No debería haber más elementos en la página web, pero el método == puede fallar si hay un error en la web.
    return True # Si no se encuentra la clase, asumimos que es la última página para evitar errores en el bucle de paginación.

In [11]:
#Uso de la función para verificar
ultima_pagina = bool_is_final_page(Extraccion_HTML_dinamico, etiquetas_a_extraer[0])
print(f"¿Es la última página? {ultima_pagina}")

¿Es la última página? False


##### **result-panel:** Contiene todos los subdominios de los articulos

In [12]:
etiqueta_result_panel = 'result-panel' # Definida en este apartado para evitar buscarla arriba en el código
result_panel = Extraccion_HTML_dinamico.find(class_=etiqueta_result_panel)  # Buscar el primer elemento con la clase 'search-stats' (solo existe uno)
if result_panel:
    print(f"Cantidad de elementos encontrados con la clase '{etiqueta_result_panel}': {len(result_panel)}")
    print("Contenido de 'search-stats':")
    print(result_panel)
    print("-----")
    for elemento in result_panel:
        print(elemento)
        print("-----")

Cantidad de elementos encontrados con la clase 'result-panel': 9
Contenido de 'search-stats':
<div class="result-panel"><div class="tile eventlandingpage theme-agave tile-grid"><div class="image-wrapper bg-agave"><img alt="Nighttime skyline in Sao Paulo Brazil " loading="eager" src="https://www.freshproduce.com/siteassets/images/events/brazil-conference/sao-paulo-skyline-at-night.jpg/_croppings/event-landing-page-grid.jpg"/></div><div class="caption"><p class="eyebrow">August 6-7, 2025 | São Paulo, Brazil</p><p class="title">The Brazil Conference &amp; Expo</p><p class="description">The Brazil Conference &amp; Expo the meeting point for retail buyers, executives &amp; industry leaders dedicated to the fresh produce supply chain in Brazil.</p><div class="cta-area"><a class="score-button seed-link" href="/events/the-brazil-conference/" target="_self">learn more</a></div></div></div><div class="tile resourcedetailpage theme-mint tile-grid"><div class="image-wrapper bg-mint"><img alt='Bask

Al inspeccionar cada elemento del articulo, cada uno contiene un botón con el hipervinculo en la clase "cta-area", por lo que ahora extraeremos el contenido solamente en "cta-area". Se buscará la etiqueta `<a>` y dentro de ella el hipervinculo.

In [13]:
etiqueta_cta_area = 'cta-area'  # Definida en este apartado para evitar buscarla arriba en el código
encontrar_todos_los_subdominios = result_panel.find_all('div', class_=etiqueta_cta_area)  # Buscar todos los hipervínculos en el HTML
print(encontrar_todos_los_subdominios)
todos_los_subdominios = []  # Lista para almacenar los subdominios encontrados
for enlace in encontrar_todos_los_subdominios:
    a_tag = enlace.find('a')  # Buscar la etiqueta <a> dentro de cada div
    #print(a_tag)
    if a_tag and 'href' in a_tag.attrs:  # Verificar si existe el atributo href
        print(f"Enlace encontrado: {a_tag['href']}")  # Imprimir el enlace
        todos_los_subdominios.append(a_tag['href'])
    else:
        print("No se encontró un enlace válido en este elemento.")

[<div class="cta-area"><a class="score-button seed-link" href="/events/the-brazil-conference/" target="_self">learn more</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/resources/consumer-trends/americans-and-sustainability/" target="_self">Learn More</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/events/the-mexico-conference/" target="_self">Learn more</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/resources/advocacy/trumps-first-100-days/" target="_self">Learn More</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/resources/global-trade/tariff-resources/" target="_self">Learn More</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/resources/Commodities/international-retail-point-of-sale-data/" target="_self">Learn More</a></div>, <div class="cta-area"><a class="score-button seed-link" href="/resources/supply-chain-management/global-market-access/" target="_se

Al utilizar el dominio principal (https://www.freshproduce.com) junto al subdominio podemos acceder a las páginas web con su respectivo contenido.

In [14]:
dominio_principal = "https://www.freshproduce.com"
subdominio_ejemplo = todos_los_subdominios[0]

print(f"Link hacia el articulo: {dominio_principal}{subdominio_ejemplo}")

Link hacia el articulo: https://www.freshproduce.com/events/the-brazil-conference/


Con ello podemos hacer una función que permita tomar desde el HTML todos los subdominios de la página web. Esto mediante encontrar la clase result-panel, extraer el contenido de cta-area para la obtención de subdominios, y añadirle el dominio principal al inicio de cada subdominio

In [15]:
def obtener_subdominios(HTML, etiqueta_completa_todos_elementos = 'result-panel' ,etiqueta_elemento_individual='cta-area', dominio_principal="https://www.freshproduce.com"):
    result_panel_subdominio = HTML.find(class_=etiqueta_completa_todos_elementos) 
    
    if not result_panel_subdominio:
        print(f"No se encontró la clase '{etiqueta}' en el HTML.")
        return []
    subdominios = []
    encontrar_todos_los_hipervinculos = result_panel_subdominio.find_all('div', class_=etiqueta_elemento_individual)
    for enlace in encontrar_todos_los_hipervinculos:
        a_tag = enlace.find('a')
        if a_tag and 'href' in a_tag.attrs:
            subdominio_completo = f"{dominio_principal}{a_tag['href']}"
            subdominios.append(subdominio_completo)
    
    return subdominios

In [16]:
ejemplo_subdominios = obtener_subdominios(Extraccion_HTML_dinamico)
print("Subdominios encontrados:")
for subdominio in ejemplo_subdominios:
    print(subdominio)

Subdominios encontrados:
https://www.freshproduce.com/events/the-brazil-conference/
https://www.freshproduce.com/resources/consumer-trends/americans-and-sustainability/
https://www.freshproduce.com/events/the-mexico-conference/
https://www.freshproduce.com/resources/advocacy/trumps-first-100-days/
https://www.freshproduce.com/resources/global-trade/tariff-resources/
https://www.freshproduce.com/resources/Commodities/international-retail-point-of-sale-data/
https://www.freshproduce.com/resources/supply-chain-management/global-market-access/
https://www.freshproduce.com/resources/global-trade/mexico-exports-to-us/
https://www.freshproduce.com/resources/Commodities/global-table-grape-shipment-data/


#### Navegar por el numero de paginas del sitio web.

Al usar el dominio https://www.freshproduce.com/resources/global-trade/ y navegar por la página, este cambia a https://www.freshproduce.com/resources/global-trade/?pageNumber=0 y https://www.freshproduce.com/resources/global-trade/?pageNumber=1 en su segunda página, por lo que asumimos que para cambiar de página utilizaremos esto:

`https://www.freshproduce.com/resources/global-trade/?pageNumber=NUMPAGINA`, donde NUMPAGINA es el número de página (iniciando desde 0).

Esto aplica para las demás categorías.

URLS con su respectiva navegación:
- https://www.freshproduce.com/resources/global-trade/?pageNumber=0
- https://www.freshproduce.com/resources/technology/?pageNumber=0
- https://www.freshproduce.com/resources/food-safety/?pageNumber=0

Por lo que el algoritmo para usarlo es:
`${URL_BASE}?pageNumber=${NUMPAGINA}` donde por ejemplo la URL base de global-trade es `https://www.freshproduce.com/resources/global-trade/` y NUMPAGINA es el número de página

In [17]:
numero_de_paginas_maximo_a_extraer = 1  # Número máximo de páginas a extraer
paginas_a_extraer = []  # Lista para almacenar las URLs de las páginas a extraer
for i in range (0,numero_de_paginas_maximo_a_extraer):
    print(f"Extrayendo página {i + 1} de {numero_de_paginas_maximo_a_extraer}")
    url_pagina = f"{primera_url_a_extraer_subdominio_ejemplo}?pageNumber={i}"
    print(f"URL de la página a extraer: {url_pagina}")
    Extraccion_HTML_dinamico = scrape_website_with_dynamic_content(url_pagina, classes_to_wait=etiquetas_a_extraer, ID_OR_CLASS='CLASS')
    
    subdominios_en_pagina = obtener_subdominios(Extraccion_HTML_dinamico)
    for subdominio in subdominios_en_pagina:
        print(subdominio)
        paginas_a_extraer.append(subdominio)  # Agregar la URL de la página a la lista

    if bool_is_final_page(Extraccion_HTML_dinamico, etiquetas_a_extraer[0]):
        print("Se ha alcanzado la última página. Terminando la extracción.")
        break

Extrayendo página 1 de 1
URL de la página a extraer: https://www.freshproduce.com/resources/global-trade/?pageNumber=0
https://www.freshproduce.com/events/the-brazil-conference/
https://www.freshproduce.com/resources/consumer-trends/americans-and-sustainability/
https://www.freshproduce.com/events/the-mexico-conference/
https://www.freshproduce.com/resources/advocacy/trumps-first-100-days/
https://www.freshproduce.com/resources/global-trade/tariff-resources/
https://www.freshproduce.com/resources/Commodities/international-retail-point-of-sale-data/
https://www.freshproduce.com/resources/supply-chain-management/global-market-access/
https://www.freshproduce.com/resources/global-trade/mexico-exports-to-us/
https://www.freshproduce.com/resources/Commodities/global-table-grape-shipment-data/


In [18]:
print(paginas_a_extraer)

['https://www.freshproduce.com/events/the-brazil-conference/', 'https://www.freshproduce.com/resources/consumer-trends/americans-and-sustainability/', 'https://www.freshproduce.com/events/the-mexico-conference/', 'https://www.freshproduce.com/resources/advocacy/trumps-first-100-days/', 'https://www.freshproduce.com/resources/global-trade/tariff-resources/', 'https://www.freshproduce.com/resources/Commodities/international-retail-point-of-sale-data/', 'https://www.freshproduce.com/resources/supply-chain-management/global-market-access/', 'https://www.freshproduce.com/resources/global-trade/mexico-exports-to-us/', 'https://www.freshproduce.com/resources/Commodities/global-table-grape-shipment-data/']


Con todo lo anterior podemos realizar una función que realice una extracción de cada página dominio junto a los dominios de cada artículo.

In [19]:
def extraccion_de_subdominios(url_a_extraer, numero_de_paginas_maximo_a_extraer=1, etiquetas_a_extraer_funcion=['search-stats', 'result-panel'], etiqueta_elemento_individual='cta-area', dominio_principal="https://www.freshproduce.com"):
    paginas_a_extraer = []  # Lista para almacenar las URLs de las páginas a extraer
    for (url, etiqueta) in url_a_extraer: #Separacion de URL y etiqueta para la extracción
        for i in range(0,numero_de_paginas_maximo_a_extraer):
            url_pagina = f"{url}?pageNumber={i}"
            print(f"URL de la página actual a extraer: {url_pagina}")
            Extraccion_HTML_dinamico = scrape_website_with_dynamic_content(url_pagina, classes_to_wait=etiquetas_a_extraer_funcion, ID_OR_CLASS='CLASS')

            etiqueta_completa_todos_elementos = etiquetas_a_extraer_funcion[1]
            subdominios_en_pagina = obtener_subdominios(Extraccion_HTML_dinamico, etiqueta_completa_todos_elementos,etiqueta_elemento_individual, dominio_principal)
            for subdominio in subdominios_en_pagina:
                #print(subdominio)
                paginas_a_extraer.append([subdominio, etiqueta])  # Agregar la URL de la página a la lista y su etiqueta asociada
            etiqueta_busqueda_numeroElementos = etiquetas_a_extraer_funcion[0]
              
            # Verificar si se ha alcanzado la última página
            if bool_is_final_page(Extraccion_HTML_dinamico, etiqueta_busqueda_numeroElementos):
                print("Se ha alcanzado la última página. Terminando la extracción.")
                break
    print("Páginas extraidas:", len(paginas_a_extraer))
    return paginas_a_extraer

In [20]:
urls_extraer_información = [
                            ["https://www.freshproduce.com/resources/global-trade/","Global Trade"],
                            ["https://www.freshproduce.com/resources/technology/","Technology"],
                            ["https://www.freshproduce.com/resources/food-safety/","Food Safety"]
                            ] # Ya viene con su respectivo etiquetado para identificar el subdominio

In [21]:
num_paginas_extraer = 1 # Solo se ocupa extraer una página, la primera página.
paginas_extraidas = extraccion_de_subdominios(urls_extraer_información, num_paginas_extraer)

URL de la página actual a extraer: https://www.freshproduce.com/resources/global-trade/?pageNumber=0
URL de la página actual a extraer: https://www.freshproduce.com/resources/technology/?pageNumber=0
URL de la página actual a extraer: https://www.freshproduce.com/resources/food-safety/?pageNumber=0
Páginas extraidas: 27


In [22]:
print(f"Total de subdominios extraídos: {len(paginas_extraidas)}")

Total de subdominios extraídos: 27


Ahora lo pasamos a un formato Pandas para mejor control de los datos

In [23]:
import pandas as pd
columnas = ['URL','Category']
df_dominios = pd.DataFrame(paginas_extraidas, columns=columnas)

In [24]:
print(df_dominios.head())

                                                 URL      Category
0  https://www.freshproduce.com/events/the-brazil...  Global Trade
1  https://www.freshproduce.com/resources/consume...  Global Trade
2  https://www.freshproduce.com/events/the-mexico...  Global Trade
3  https://www.freshproduce.com/resources/advocac...  Global Trade
4  https://www.freshproduce.com/resources/global-...  Global Trade


In [25]:
print(df_dominios.Category.describe()) #Ver descripción de las categorías

count               27
unique               3
top       Global Trade
freq                 9
Name: Category, dtype: object


In [26]:
df_dominios.to_csv(f"extracted_subdomains.csv", index=False)  # Guardar el DataFrame en un archivo CSV

### Extracción de información de artículos.

Teniendo la información de los anteriores subdominios en un .csv vamos a extraer toda la información de cada página web.

In [14]:
import pandas as pd
df_subdominios = pd.read_csv("extracted_subdomains.csv")  # Cargar el DataFrame desde el archivo CSV
df_subdominios.head()  # Mostrar las primeras filas del DataFrame

,URL,Category
0,https://www.freshproduce.com/events/the-brazil...,Global Trade
1,https://www.freshproduce.com/resources/consume...,Global Trade
2,https://www.freshproduce.com/events/the-mexico...,Global Trade
3,https://www.freshproduce.com/resources/advocac...,Global Trade
4,https://www.freshproduce.com/resources/global-...,Global Trade


Ciclo for para ver todo el contenido

In [15]:
for i in range(len(df_subdominios)):
    URL, Category = df_subdominios.iloc[i][['URL', 'Category']]  # Acceder a cada fila por índice
    print(f"URL: {URL}, Category: {Category}")

URL: https://www.freshproduce.com/events/the-brazil-conference/, Category: Global Trade
URL: https://www.freshproduce.com/resources/consumer-trends/americans-and-sustainability/, Category: Global Trade
URL: https://www.freshproduce.com/events/the-mexico-conference/, Category: Global Trade
URL: https://www.freshproduce.com/resources/advocacy/trumps-first-100-days/, Category: Global Trade
URL: https://www.freshproduce.com/resources/global-trade/tariff-resources/, Category: Global Trade
URL: https://www.freshproduce.com/resources/Commodities/international-retail-point-of-sale-data/, Category: Global Trade
URL: https://www.freshproduce.com/resources/supply-chain-management/global-market-access/, Category: Global Trade
URL: https://www.freshproduce.com/resources/global-trade/mexico-exports-to-us/, Category: Global Trade
URL: https://www.freshproduce.com/resources/Commodities/global-table-grape-shipment-data/, Category: Global Trade
URL: https://www.freshproduce.com/resources/technology/take

In [16]:
primer_ejemplo_url_a_extraer_informacion_completa = df_subdominios.iloc[0]['URL']  # Obtener la primera URL del DataFrame
print(primer_ejemplo_url_a_extraer_informacion_completa)

https://www.freshproduce.com/events/the-brazil-conference/


Volvemos a colocar la función de Web scrapping para contenido dinámico, para evitar correr el proceso anterior

In [17]:
from selenium import webdriver #Controlar google chrome para obtener HTML con contenido dinámico
from selenium.webdriver.chrome.options import Options #Controlar google chrome sin interfaz gráfica
from selenium.webdriver.support.ui import WebDriverWait  # Para esperar elementos dinámicos
from selenium.webdriver.support import expected_conditions as EC  # Condiciones para la espera
from selenium.webdriver.common.by import By  # Para localizar elementos (por clase, ID, etc.)
from bs4 import BeautifulSoup # #Para parsear el HTML obtenido con selenium
import chromedriver_autoinstaller #Instalación automática del driver de Chrome

def scrape_website_with_dynamic_content(page, classes_to_wait = None, ID_OR_CLASS='CLASS'):
    # Instala automáticamente el chromedriver compatible con la versión de Chrome instalada
    chromedriver_autoinstaller.install() 

    options = Options() # Configuraciones de opciones para el navegador
    options.add_argument("--headless") # Ejecuta chrome sin interfaz gráfica

    #Necesario por falta de permisos en el entorno aislado del contenedor
    options.add_argument("--no-sandbox") #Desactiva seguridad de Chrome por problemas de permisos en algunos sistemas
    
    options.add_argument("--disable-dev-shm-usage") #Prevenir fallos por falta de memoria (RAM) compartida en algunos sistemas. Usa memoria de disco duro.

    driver = webdriver.Chrome(options=options) # Inicia navegador chrome con las opciones configuradas

    driver.get(page) # Carga la página web en el navegador

    
    if classes_to_wait:
        tiempo_espera = 30 # Tiempo máximo de espera en segundos para que se cargue el contenido dinámico
        wait = WebDriverWait(driver, tiempo_espera)
        if ID_OR_CLASS.upper() == 'CLASS':
            for class_to_wait in classes_to_wait: #Recorre cada clase a esperar
                wait.until(
                    EC.presence_of_element_located((By.CLASS_NAME, class_to_wait)) #CAMBIO: Cambié By.CLASS_NAME por By.ID para esperar por ID en lugar de clase
                )# Espera cada una de las clases
        elif ID_OR_CLASS.upper() == 'ID':
            for id_to_wait in classes_to_wait: #Recorre cada ID a esperar
                wait.until(
                    EC.presence_of_element_located((By.ID, id_to_wait)) #CAMBIO: Cambié By.ID por By.CLASS_NAME para esperar por clase en lugar de ID
                )# Espera cada uno de los IDs
        else:
            raise ValueError("ID_OR_CLASS debe ser 'CLASS' o 'ID'.")  # Manejo de error si el parámetro no es válido



    html_completo = driver.page_source  # Obtención del HTML renderizado completo

    # Usar BeautifulSoup para parsear
    soup = BeautifulSoup(html_completo, "html.parser")
    # Por ejemplo, buscar la tabla con clase 'search-results'
    driver.quit() #Cerrar navegador de Chrome
    return soup

Inspeccionado la página web, lo que se requiere en lugar de una clase será un identificador id llamado "pageContent"

In [18]:
Extraccion_HTML_dinamico_todo_el_contenido = scrape_website_with_dynamic_content(primer_ejemplo_url_a_extraer_informacion_completa, 
                                                                                 classes_to_wait=['pageContent'], ID_OR_CLASS='ID')

Se requiere el titulo de la página. Esta es la etiqueta h1 de la página

In [19]:
titulo = Extraccion_HTML_dinamico_todo_el_contenido.find('h1')  # Buscar el título de la página
print(f"Título de la página: {titulo.text.strip() if titulo else 'No se encontró el título'}")

Título de la página: The Brazil Conference


Extraer todo el contenido de la página

In [20]:
all_content_page = Extraccion_HTML_dinamico_todo_el_contenido.find(id='pageContent')  # Buscar el contenido principal de la página. Evitamos barras como cookies, banners, etc.
if all_content_page:
    print("Contenido de la página:")
    print(all_content_page.text)

Contenido de la página:









August 6-7, 2025 | São Paulo, Brazil
The Brazil Conference


THE BRAZIL CONFERENCE & EXPO is the meeting point for the main retail buyers, high-level production executives and industry leaders at the only event in Brazil dedicated to the entire fresh produce supply chain.




    Register now




















IFPA Home





Events





The Brazil Conference









Share


Tweet


Email


Share


























Why Attend?

Find innovation. Strengthen business relationships. Network with industry leaders and stay on top of industry trends and news.









Who Attends?

Retailers, producers, wholesalers, importers, exporters, industry associations, government agencies, solution and service providers












2024 Brazil Conference Photos





Business, networking, and innovation! Discover the latest trends in the fruit, flower, vegetable, and produce market at The Produce Fresh International.





From farm to table! The Produce Fresh I

Muchos espacios en blanco. Esto puede afectar en el número de Tokens que puede recibir la LLM. Hay que reducirlo

In [21]:
string_prueba = """
Contenido de la página:









August 6-7, 2025 | São Paulo, Brazil
The Brazil Conference


THE BRAZIL CONFERENCE & EXPO is the meeting point for the main retail buyers, high-level production executives and industry leaders at the only event in Brazil dedicated to the entire fresh produce supply chain.




    Register now
"""

#print(string_prueba.strip())  # Eliminar espacios en blanco al principio y al final del string
string_prueba_sin_espacios_inicio_y_final = string_prueba.strip()  # Eliminar espacios en blanco al principio y al final del string
texto = ""
for line in string_prueba_sin_espacios_inicio_y_final.split('\n'): #Hace un arreglo de saltos de linea
    line = line.strip() # Eliminar espacios en blanco al principio y al final de cada línea
    if line:  # Verificar si la línea no está vacía
        texto += line + "\n"  # Concatenar la línea al texto final, agregando un espacio al final

print("Texto final:")
print(texto.strip())  # Imprimir el texto final después de eliminar espacios en blanco al principio y al final


Texto final:
Contenido de la página:
August 6-7, 2025 | São Paulo, Brazil
The Brazil Conference
THE BRAZIL CONFERENCE & EXPO is the meeting point for the main retail buyers, high-level production executives and industry leaders at the only event in Brazil dedicated to the entire fresh produce supply chain.
Register now


Podemos hacer una función para limpieza de espacios en blanco

In [22]:
def clean_text(text):
    # Elimina espacios en blanco al principio y al final, y elimina líneas vacías
    text_without_empty_lines_start_end = text.strip()  # Eliminar espacios en blanco al principio y al final del texto 
    cleaned_text = ""
    for line in text_without_empty_lines_start_end.split('\n'):  # Hace un arreglo de saltos de línea
        line = line.strip()  # Eliminar espacios en blanco al principio y al final de cada línea
        if line:  # Verificar si la línea no está vacía
            cleaned_text += line + "\n"  # Concatenar la línea al texto final, agregando un salto de línea al final
    return cleaned_text.strip()  # Devolver el texto final después de eliminar espacios en blanco al principio y al final


In [23]:
clean_text_final = clean_text(all_content_page.text)

In [24]:
print(clean_text_final)  # Imprimir el texto limpio

August 6-7, 2025 | São Paulo, Brazil
The Brazil Conference
THE BRAZIL CONFERENCE & EXPO is the meeting point for the main retail buyers, high-level production executives and industry leaders at the only event in Brazil dedicated to the entire fresh produce supply chain.
Register now
IFPA Home
Events
The Brazil Conference
Share
Tweet
Email
Share
Why Attend?
Find innovation. Strengthen business relationships. Network with industry leaders and stay on top of industry trends and news.
Who Attends?
Retailers, producers, wholesalers, importers, exporters, industry associations, government agencies, solution and service providers
2024 Brazil Conference Photos
Business, networking, and innovation! Discover the latest trends in the fruit, flower, vegetable, and produce market at The Produce Fresh International.
From farm to table! The Produce Fresh International is the perfect platform to connect producers and consumers.
Kitchen inspiration! Discover new ideas and recipes with fresh products fr

Hacemos una función que al recibir un pandas, extraiga la información del contentpage de cada URL y a su vez el título de la página

In [25]:
def copy_df_and_add_title_and_text_from_URL(dataframe):
    # Extrae el texto de la columna 'URL' del DataFrame y lo coloca en una nueva columna 'Text'
    dataframe_result = dataframe.copy()  # Hacer una copia del DataFrame original para evitar modificarlo directamente
    dataframe_result['Title'] = None  # Crear una nueva columna 'Title' con valores nulos
    dataframe_result['FullArticleText'] = None  # Crear una nueva columna 'Topics' con valores nulos
    
    for i in range(len(dataframe_result)):
        URL, Category = dataframe_result.iloc[i][['URL', 'Category']]  # Acceder a cada fila por índice
        print(f"Extrayendo texto de la URL: {URL}, Categoría: {Category}")
        id_a_extraer = 'pageContent'  # ID del elemento HTML que contiene el texto
        Extraccion_HTML_dinamico_todo_el_contenido = scrape_website_with_dynamic_content(URL, classes_to_wait=[id_a_extraer], ID_OR_CLASS='ID')
        title_page = Extraccion_HTML_dinamico_todo_el_contenido.find('h1')  # Buscar el título de la página
        if title_page:
            dataframe_result.at[i, 'Title'] = title_page.text.strip()  # Asignar el título a una nueva columna 'Title'
        else:
            print(f"No se encontró el título en la URL: {URL}")
            dataframe_result.at[i, 'Title'] = None
        all_content_page = Extraccion_HTML_dinamico_todo_el_contenido.find(id=id_a_extraer)  # Buscar el contenido principal de la página
        if all_content_page:
            cleaned_text = clean_text(all_content_page.text)
            dataframe_result.at[i, 'FullArticleText'] = cleaned_text  # Asignar el texto limpio a la nueva columna
        else:
            print(f"No se encontró el contenido principal en la URL: {URL}")
            dataframe_result.at[i, 'FullArticleText'] = None
    dataframe_result = dataframe_result[['Title', 'URL', 'Category', 'FullArticleText']]  # Reordenar las columnas del DataFrame
    return dataframe_result  # Devolver el DataFrame con la nueva columna 'Text' que contiene el texto extraído de las URLs

In [26]:
dataframe_nuevo = copy_df_and_add_title_and_text_from_URL(df_subdominios)  # Llamar a la función para extraer el texto de las URLs y crear un nuevo DataFrame

Extrayendo texto de la URL: https://www.freshproduce.com/events/the-brazil-conference/, Categoría: Global Trade
Extrayendo texto de la URL: https://www.freshproduce.com/resources/consumer-trends/americans-and-sustainability/, Categoría: Global Trade
Extrayendo texto de la URL: https://www.freshproduce.com/events/the-mexico-conference/, Categoría: Global Trade
Extrayendo texto de la URL: https://www.freshproduce.com/resources/advocacy/trumps-first-100-days/, Categoría: Global Trade
Extrayendo texto de la URL: https://www.freshproduce.com/resources/global-trade/tariff-resources/, Categoría: Global Trade
Extrayendo texto de la URL: https://www.freshproduce.com/resources/Commodities/international-retail-point-of-sale-data/, Categoría: Global Trade
Extrayendo texto de la URL: https://www.freshproduce.com/resources/supply-chain-management/global-market-access/, Categoría: Global Trade
Extrayendo texto de la URL: https://www.freshproduce.com/resources/global-trade/mexico-exports-to-us/, Categ

In [27]:
dataframe_nuevo.to_csv("scraped_data.csv", index=False)  # Guardar el DataFrame en un archivo CSV
print("Datos extraídos y guardados en 'scraped_data.csv'.")

Datos extraídos y guardados en 'scraped_data.csv'.


In [28]:
print(dataframe_nuevo.describe())

                                    Title  \
count                                  27   
unique                                 26   
top     Americans & Sustainable Practices   
freq                                    2   

                                                      URL      Category  \
count                                                  27            27   
unique                                                 26             3   
top     https://www.freshproduce.com/resources/consume...  Global Trade   
freq                                                    2             9   

                                          FullArticleText  
count                                                  27  
unique                                                 26  
top     Consumer Trends\nAmericans & Sustainable Pract...  
freq                                                    2  


## AI-Powered Analysis & Enrichment

En este apartado usaremos una LLM, mediante una interface API

### Carga de datos del proceso anterior

In [14]:
import pandas as pd
df_scraped_data = pd.read_csv("scraped_data.csv")  # Cargar el DataFrame desde el archivo CSV
(df_scraped_data.head())  # Mostrar las primeras filas del DataFrame

,Title,URL,Category,FullArticleText
0,The Brazil Conference,https://www.freshproduce.com/events/the-brazil...,Global Trade,"August 6-7, 2025 | São Paulo, Brazil\nThe Braz..."
1,Americans & Sustainable Practices,https://www.freshproduce.com/resources/consume...,Global Trade,Consumer Trends\nAmericans & Sustainable Pract...
2,The Mexico Conference,https://www.freshproduce.com/events/the-mexico...,Global Trade,"May 14-15, 2025 | Guadalajara, Mexico\nThe Mex..."
3,Trump’s First 100 Days,https://www.freshproduce.com/resources/advocac...,Global Trade,Webinar\nTrump’s First 100 Days\nWhat It Means...
4,Tariff Resources,https://www.freshproduce.com/resources/global-...,Global Trade,Global Trade\nTariff Resources\nImpact of Tari...


### Descarga del modelo LLM

Al correr la red docker-compose, se corre un contenedor con Ollama. Se puede acceder a modelos LLM descargados desde un repositorio de LLM (https://ollama.com/search) y mediante su apartado de API (https://github.com/ollama/ollama/blob/main/docs/api.md) podemos descargar el modelo y/o hacer peticiones mediante prompts

In [15]:
import json
import requests
from dotenv import load_dotenv
import os
load_dotenv()

URL_OLLAMA = os.getenv("URL_OLLAMA") #Cargamos un archivo .env con la URL de Ollama
print(f"URL de Ollama: {URL_OLLAMA}")

URL de Ollama: http://localhost:11434


Descargamos el modelo con la siguiente función.

In [16]:
def pull(model):
    #URL por defecto para descargas de modelos LLM
    url = f'{URL_OLLAMA}/api/pull'
    # Comprobar que los valores obligatorios están presentes
    response_data = {
        "model": model,
        "stream": False #Evita usar streaming. Que no mande la información en partes y la mande en su lugar todo en conjunto
    }
    try:
        #Usa POST para enviar los datos al servidor
        response = requests.post(url, json=response_data)
        if response.status_code == 200:
            return response.json(), 200
        else:
            print("Error:", response.status_code, response.text)
            return {"error": f"Error {response.status_code}: {response.text}"}, response.status_code
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)
        return {"error": f"An error occurred: {e}"}, 500

Descargamos el modelo Llama3.2

In [17]:
model_pull = "llama3.2"

response = pull(model_pull)
print(response)

({'status': 'success'}, 200)


### Envío de promps a una LLM local mediante Ollama y su API

Ollama ofrece una interfaz que permite hacer consultas a una LLM descargada, esto mediante bibliotecas como ollama-python, o mediante su API. Para este ejercicio usaremos la API de Ollama

Mediante esta función podemos realizar consultas a un modelo LLM local.
- El modelo LLM debe de estar previamente descargado mediante el paso "Descarga del modelo LLM"
- Se puede usar un formato JSON schema que el modelo puede entregar como respuesta
- se puede mantener vivo por X cantidad de minutos mediante keep_alive (-1 es por siempre, y 0 detiene la LLM en memoria)

In [18]:
def generate(model,prompt = None, format = None, keep_alive = None):
    url = f'{URL_OLLAMA}/api/generate'
    # Comprobar que los valores obligatorios están presentes
    response_data = {
        "model": model,
        "stream": False #Evita usar streaming. Que no mande la información en partes y la mande en su lugar todo en conjunto
    }
    #Corroborar los valores opcionales y los añade en dado caso de existir
    if prompt:
        response_data["prompt"] = prompt
    if format:
        response_data["format"] = format
    if keep_alive:
        response_data["keep_alive"] = keep_alive
    try:
        response = requests.post(url, json=response_data)
        if response.status_code == 200:
            return response.json()['response'], 200 #Devolver código HTTP junto a la respuesta del modelo
        else:
            print("Error:", response.status_code, response.text)
            return {"error": f"Error {response.status_code}: {response.text}"}, response.status_code # Devolver error si la solicitud no fue exitosa junto con el código HTTP correspondiente
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)
        return {"error": f"An error occurred: {e}"}, 500 # Devolver error si ocurre un problema con la solicitud HTTP junto con el código HTTP 500

### Rubricas en Ollama
Ollama acepta rúbricas del tipo JSON schema, por lo tanto usaremos la siguiente rúbrica para el topic extraction:

In [19]:
format = {
    "type": "object",
    "properties": {
      "response": {
        "type": "array",
        "minItems": 3,
        "maxItems": 5,
        "items": {
          "type": "string"
        }
      }
    },
    "required": [
      "response"
    ]
  }

"""
Respuesta entregada de ejemplo: {

  "response": [
    "The Brazil Conference & Expo",
    "International Fresh Produce Association",
    "COP30",
    "Brazil Sustainability Leadership"
  ]
}
   
Codigo de respuesta: 200
"""

'\nRespuesta entregada de ejemplo: {\n\n  "response": [\n    "The Brazil Conference & Expo",\n    "International Fresh Produce Association",\n    "COP30",\n    "Brazil Sustainability Leadership"\n  ]\n}\n   \nCodigo de respuesta: 200\n'

### Prompt para el modelo.

En este apartado se hace la propuesta de prompt para los dos incisos descritos en el documento

#### Primer inciso

Realizamos el prompt para el primer inciso:

*Summarization: Generate a concise, one-sentence summary of the article's main point.*

- Propuesta: This is a article about the category **VAR_CATEGORY** with the title '**VAR_TITLE**'.
Generate a concise, one-sentence summary of the article's main point.
The article is as follows:
**VAR_FULL_ARTICLE_TEXT**

In [20]:
category_example_1 = "Global Trade"  # Categoría del artículo
title_example_1 = "The Brazil Conference & Expo"  # Título del artículo
full_article_text_example_1 = """
"This is an article about brazil"
"""
prompt_first_example= f"""
This is a article about the category {category_example_1} with the title '{title_example_1}'.
Generate a concise, one-sentence summary of the article's main point.
The article is as follows:
{full_article_text_example_1} 
    """

#### Segundo inciso

Realizamos el prompt para el segundo inciso:

*Topic Extraction: Identify and list 3-5 primary topics or keywords from the text (e.g., ["Avocado Trade", "Supply Chain", "AI in Agriculture", "Foodborne Illness"]).*

- Propuesta: This is a article about the category **VAR_CATEGORY** with the title '**VAR_TITLE**'.
Identify and list 3-5 primary topics or keywords from the text.
RESPOND USING A JSON FORMAT.
The article is as follows:
**VAR_FULL_ARTICLE_TEXT**

Para la rúbrica utilizaremos structured outputs de Ollama (https://ollama.com/blog/structured-outputs)

Utilizaremos la siguiente rúbrica para dar un resultado similar a este: ["Avocado Trade", "Supply Chain", "AI in Agriculture", "Foodborne Illness"]

In [21]:
category_example_2 = "Global Trade"  # Categoría del artículo
title_example_2 = "The Brazil Conference & Expo"  # Título del artículo
full_article_text_example_2 = """
"This is an article about brazil"
"""
prompt_first_example= f"""
This is a article about the category {category_example_1} with the title '{title_example_1}'.
Identify and list 3-5 primary topics or keywords from the text.
RESPOND USING A JSON FORMAT.
The article is as follows:
{full_article_text_example_1} 
"""

format_a_usar = {
    "type": "object",
    "properties": {
      "response": {
        "type": "array",
        "minItems": 3, #Numero de elementos mínimo a devolver
        "maxItems": 5, #Numero de elementos máximo a devolver
        "items": {
          "type": "string"
        }
      }
    },
    "required": [
      "response"
    ]
  }

  # Formato a usar para la respuesta del modelo, visto en el apartado de rubricas en Ollama

La razón de un JSON en lugar de un arreglo es porque es un formato más controlado que Ollama nos ofrece, y que tiene más robustés de ofrecernos sin que ofrezca resultados extraños.

Para acceder al resultado deseado, solo es cuestión de acceder al valor del JSON response

### Consultas con la información en el Dataframe
Se realiza una función que automatice los pasos anteriores, para añadir la respectiva respuesta a una columna *Summary* y a una columna *Topics* del dataframe obtenido en webScrapping

Para la función automatizada realizamos los siguientes pasos:
1. Copia del Dataframe (para no alterar el original)
2. Añadido columna "Summary" y "Topics" con valores None
3. Ciclo for para pasar todos los elementos
4. Cada elemento hace una consulta para la primera y segunda petición
5. Si ambas dan código 200 y pasan, se guardan en el dataframe copia
6. Si algo da error, se guarda como None

In [22]:
def generate_summary_and_topics_with_llm(dataframe, modelo="llama3.2", format=None):
    if format is None: #Formato por defecto si no se especifica
        format = {
    "type": "object",
    "properties": {
      "response": {
        "type": "array",
        "minItems": 3,
        "maxItems": 5,
        "items": {
          "type": "string"
        }
      }
    },
    "required": [
      "response"
    ]
  }
    dataframe_with_llm_responses = dataframe.copy()  # Hacer una copia del DataFrame original para evitar modificarlo directamente
    dataframe_with_llm_responses['Summary'] = None  # Crear una nueva columna 'Summary' para almacenar las respuestas del LLM
    dataframe_with_llm_responses['Topics'] = None  # Crear una nueva columna 'Topics' para almacenar las respuestas del LLM
    for i in range(len(dataframe_with_llm_responses)):
        title = dataframe_with_llm_responses.iloc[i]['Title']  # Obtener el título de la fila actual
        url = dataframe_with_llm_responses.iloc[i]['URL']  # Obtener la URL de la fila actual
        category = dataframe_with_llm_responses.iloc[i]['Category']
        full_article_text = dataframe_with_llm_responses.iloc[i]['FullArticleText']
        print(f"Article Number {i + 1} of {len(dataframe_with_llm_responses)}")

        prompt_first= f"""
    This is a article about the category {category} with the title '{title}'.
    Generate a concise, one-sentence summary of the article's main point.
    The article is as follows:
    {full_article_text} 
        """

        prompt_second = f"""
    This is a article about the category {category} with the title '{title}'.
    Identify and list 3-5 primary topics or keywords from the text.
    RESPOND USING A JSON FORMAT.
    The article is as follows:
    {full_article_text}
        """
        #"""
        
        response_first, code = generate(modelo, prompt=prompt_first, format=None, keep_alive=False)
        if code == 200:
            dataframe_with_llm_responses.at[i, 'Summary'] = response_first  # Asignar None si hay error
        else:
            print("Error al generar la respuesta del LLM para la primera solicitud. URL:", url)
            dataframe_with_llm_responses.at[i, 'Summary'] = None  # Asignar la respuesta del LLM a la columna 'Summary'
        #"""
        response_second, code = generate(modelo, prompt=prompt_second, format=format, keep_alive=False)
        if code == 200:
            response_second_in_json = json.loads(response_second)  # Convertir la respuesta a JSON
            response_second_just_response = response_second_in_json['response']  # Extraer el campo 'response' del JSON
            dataframe_with_llm_responses.at[i, 'Topics'] = response_second_just_response  # Asignar None si hay error
        else:
            print("Error al generar la respuesta del LLM para la segunda solicitud. URL: ",url)
            dataframe_with_llm_responses.at[i, 'Topics'] = None  # Asignar la respuesta del LLM a la columna 'Topics'
    return dataframe_with_llm_responses  # Devolver el DataFrame con las nuevas columnas 'Summary' y 'Topics'



In [23]:
modelo = "llama3.2"  # Modelo LLM a usar
dataframe_a_usar = df_scraped_data  # DataFrame con los artículos a procesar
formato = format = {
    "type": "object",
    "properties": {
      "response": {
        "type": "array",
        "minItems": 3,
        "maxItems": 5,
        "items": {
          "type": "string"
        }
      }
    },
    "required": [
      "response"
    ]
  }
dataframe_with_llm_responses = generate_summary_and_topics_with_llm(dataframe_a_usar, modelo=modelo, format=formato)  # Llamar a la función para generar las respuestas del LLM

Article Number 1 of 27
Article Number 2 of 27
Article Number 3 of 27
Article Number 4 of 27
Article Number 5 of 27
Article Number 6 of 27
Article Number 7 of 27
Article Number 8 of 27
Article Number 9 of 27
Article Number 10 of 27
Article Number 11 of 27
Article Number 12 of 27
Article Number 13 of 27
Article Number 14 of 27
Article Number 15 of 27
Article Number 16 of 27
Article Number 17 of 27
Article Number 18 of 27
Article Number 19 of 27
Article Number 20 of 27
Article Number 21 of 27
Article Number 22 of 27
Article Number 23 of 27
Article Number 24 of 27
Article Number 25 of 27
Article Number 26 of 27
Article Number 27 of 27


In [34]:
print(dataframe_with_llm_responses.Summary[0])
print(dataframe_with_llm_responses.Topics[0])

The Brazil Conference is a major event in the global trade of fresh produce, where industry leaders and buyers gather to network, innovate, and stay informed on market trends and sustainability.
['Global Trade', 'Brazil Conference', 'Fresh Produce Supply Chain']


In [35]:
dataframe_with_llm_responses.to_csv("analysis_summary.csv", index=False)  # Guardar el DataFrame en un archivo CSV

### Verificación de valores No None en el csv final
Cargamos el CSV para verificar valores null

In [36]:
import pandas as pd
df_analysis = pd.read_csv("analysis_summary.csv")  # Cargar el DataFrame desde el archivo CSV
print(df_analysis.loc[df_analysis.Summary.isnull()])  # Mostrar las primeras filas del DataFrame
print("\n")
print(df_analysis.loc[df_analysis.Topics.isnull()])  # Mostrar las primeras filas del DataFrame

Empty DataFrame
Columns: [Title, URL, Category, FullArticleText, Summary, Topics]
Index: []


Empty DataFrame
Columns: [Title, URL, Category, FullArticleText, Summary, Topics]
Index: []


#### Ejemplo de resultado que debe de dar si existen valores null

Creación de Tabla con valores None o null

In [38]:
columns_name_example_null = ['Title', 'URL', 'Category', 'FullArticleText', 'Summary', 'Topics']  # Nombres de las columnas del DataFrame
df_with_null = df_analysis[columns_name_example_null]  # Crear un nuevo DataFrame con las columnas especificadas
df_with_null.loc[0] = ("None","None","None","None",None,"None")  # Agregar las filas donde el título es nulo al nuevo DataFrame
df_with_null.loc[1] = ("None","None","None","None",None,None)  # Agregar las filas donde el título es nulo al nuevo DataFrame
df_with_null.loc[2] = ("None","None","None","None","None",None)  # Agregar las filas donde el título es nulo al nuevo DataFrame
df_with_null.loc[3] = ("None","None","None","None",None,"None")  # Agregar las filas donde el título es nulo al nuevo DataFrame
df_with_null.loc[4] = ("None","None","None","None","None",None)  # Agregar las filas donde el título es nulo al nuevo DataFrame

In [39]:
print(df_with_null.loc[df_with_null.Summary.isnull()])  # Mostrar valores nulos en la columna 'Summary'
print("\n")
print(df_with_null.loc[df_with_null.Topics.isnull()])  # Mostrar valores nulos en la columna 'Topics'

  Title   URL Category FullArticleText Summary Topics
0  None  None     None            None    None   None
1  None  None     None            None    None   None
3  None  None     None            None    None   None


  Title   URL Category FullArticleText Summary Topics
1  None  None     None            None    None   None
2  None  None     None            None    None   None
4  None  None     None            None    None   None


## Analisis de datos

In [1]:
import pandas as pd
df_analysis = pd.read_csv("analysis_summary.csv")  # Cargar el DataFrame desde el archivo CSV
print( "Categorias:",df_analysis.Category.unique())
print("Columnas de la tabla:", df_analysis.columns)#Imprime las columnas que contiene el dataframe

Categorias: ['Global Trade' 'Technology' 'Food Safety']
Columnas de la tabla: Index(['Title', 'URL', 'Category', 'FullArticleText', 'Summary', 'Topics'], dtype='object')


### Separación por categorias

In [2]:
GT = df_analysis.loc[df_analysis.Category=='Global Trade']
TGY =  df_analysis.loc[df_analysis.Category=='Technology']
FS =  df_analysis.loc[df_analysis.Category=='Food Safety']

#### Global Trade

In [3]:
for i in range(3):
    print("URL:", GT.iloc[i].URL)  # URL del DataFrame de Global Trade
    print("Title:", GT.iloc[i].Title)  # Título del DataFrame de Global Trade
    print("Summary:",GT.iloc[i].Summary)  # Descripción del DataFrame de Global Trade
    print("Topic:",GT.iloc[i].Topics)  # Temas del DataFrame de Global Trade
    print("----")

URL: https://www.freshproduce.com/events/the-brazil-conference/
Title: The Brazil Conference
Summary: The Brazil Conference is a major event in the global trade of fresh produce, where industry leaders and buyers gather to network, innovate, and stay informed on market trends and sustainability.
Topic: ['Global Trade', 'Brazil Conference', 'Fresh Produce Supply Chain']
----
URL: https://www.freshproduce.com/resources/consumer-trends/americans-and-sustainability/
Title: Americans & Sustainable Practices
Summary: Here is a concise, one-sentence summary of the article's main point:

A U.S. consumer survey reveals that while Americans are increasingly concerned about sustainability in produce, mixed signals on terminology and packaging trade-offs hinder the adoption of eco-friendly practices, offering opportunities for clear labeling, traceable origin stories, and innovative packaging to meet consumer expectations and drive sales.
Topic: ['Americans and Sustainability', 'Sustainable Practi

#### Technology

In [4]:
for i in range(3):
    print('URL:', TGY.iloc[i].URL)  # URL del DataFrame de Global Trade
    print("Title:", TGY.iloc[i].Title)  # Título del DataFrame de Technology
    print("Summary:",TGY.iloc[i].Summary)  # Descripción del DataFrame
    print("Topic:",TGY.iloc[i].Topics)  # Temas del DataFrame de Technology
    print("----")

URL: https://www.freshproduce.com/resources/technology/takes-on-tech-podcast/episode-118-behind-the-scenes-of-produce-policy/
Title: Behind the Scenes of Produce Policy: Science and Politics
Summary: The article discusses how science, technology, and politics intersect in shaping produce policy, highlighting key areas such as labor shortages, GMOs, innovation opportunities, and public perception's influence on policy development.
Topic: ['Produce Policy', 'Science and Politics', 'Gene Editing', 'Biotechnology', 'GMOs']
----
URL: https://www.freshproduce.com/resources/technology/takes-on-tech-podcast/episode-117-the-future-of-agtech/
Title: Smart Irrigation, Disease Detection, and Solar-Powered Farming
Summary: The article discusses three innovative agricultural technologies - smart irrigation, disease detection, and solar-powered farming - that are revolutionizing the way farmers produce crops while addressing climate change and resource limitations.
Topic: ['Smart Irrigation', 'Diseas

#### Food Safety

In [5]:
for i in range(3):
    print("URL:", FS.iloc[i].URL)  # URL del DataFrame de Technology
    print("Title:", FS.iloc[i].Title)  # Título del DataFrame de Technology
    print("Summary:",FS.iloc[i].Summary)  # Descripción del DataFrame de Technology
    print("Topic:",FS.iloc[i].Topics)  # Temas del DataFrame de Technology
    print("----")

URL: https://www.freshproduce.com/resources/food-safety/risk-scoring-for-produce/
Title: Smarter Risk Scoring for Produce: Leverage Meat Industry Insights
Summary: The article provides an overview of a webinar titled "Smarter Risk Scoring for Produce: Leverage Meat Industry Insights," which aims to help produce processors adopt proven risk scoring methods from the meat industry to enhance their food safety programs.
Topic: ['Risk Scoring', 'Produce Operations', 'Data-Driven Decision Making']
----
URL: https://www.freshproduce.com/resources/consumer-trends/americans-and-sustainability/
Title: Americans & Sustainable Practices
Summary: Here is a concise, one-sentence summary of the article's main point:

A survey of 754 US consumers reveals that while many are interested in sustainable practices for produce, they face confusion and mixed signals on terminology, safety perceptions, and packaging trade-offs, highlighting opportunities for clear labeling, traceable origin stories, and innov